In [1]:
import sys

sys.path.append("../")
import os
import math
import numpy as np
import cv2
import pandas as pd
import tikzplotlib
from random import randint
import matplotlib.pyplot as plt
from pedrec.datasets.tud_dataset import TudDataset
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.models.constants.sample_method import SAMPLE_METHOD
from pedrec.configs.dataset_configs import get_tud_dataset_cfg_default
%matplotlib widget


In [2]:
cfg = PedRecNet50Config()
dataset_cfg = get_tud_dataset_cfg_default()
dataset_cfg.subsample = 2
dataset_cfg.subsampling_strategy = SAMPLE_METHOD

# MS Coco
dataset_name = "TUD"
dataset = TudDataset("data/datasets/cvpr10_multiview_pedestrians/", DatasetType.TRAIN, dataset_cfg,
                      cfg.model.input_size, None)
train_data = []
for i in range(1, 9):
  train_data += dataset.get_xml_data(os.path.join(dataset.dataset_path, f"viewpoints_train{i}.al"))

val_data = dataset.get_xml_data(os.path.join(dataset.dataset_path, f"viewpoints_validate.al"))


In [3]:
def get_df(annotations):
  bb_sizes = []
  orientations = []
  for annotation in annotations:
    bb = annotation["bb"]
    orientation = annotation["orientations"]
    bb_sizes.append(math.sqrt(bb[2]**2 + bb[3]**2))
    orientations.append(math.degrees(orientation[0][1] * 2 * math.pi))

  data = np.array([bb_sizes, orientations], dtype=np.float32).transpose()
  print(data.shape)
  return pd.DataFrame(data=data, columns=["bb_size", "orientation"])


In [37]:
def write_plot(df, name):
  fig = plt.figure()
  body_phi_deg = df['orientation']
  body_phi_deg.hist(bins=50)
  xmin, xmax, ymin, ymax = plt.axis()
  tikzplotlib.save(f"/home/dennis/Downloads/tud_{name}_body_orientation_phi.tex", extra_axis_parameters={
    'width=0.5\\textwidth',
    'title={Distribution of body $\\theta$ orientations}',
    'xlabel={degrees ($°$)}',
    'ylabel={Number of samples}',
    'enlarge x limits=0.001',
    'enlarge y limits=0.001',
    'xmin=0',
    'xmax=360',
  })

In [4]:
df_train = get_df(train_data)
df_train.describe()

(8322, 2)


,bb_size,orientation
count,8322.000000,8322.000000
mean,192.717316,176.840561
std,44.902618,103.833885
min,45.541191,0.002124
25%,188.883560,87.440254
50%,209.095673,168.580917
75%,212.906082,267.102020
max,462.312653,359.998810


In [5]:
body_phi_deg = df_train['orientation']
max_val = 0
for i in range (0, 370, 10):
  elements = body_phi_deg[body_phi_deg.between(i, i+10, "both")]
  num_els = len(elements)
  max_val = max_val if max_val > num_els else num_els
  print(f"{i},{num_els}")
print(max_val)

0,234
10,125
20,294
30,388
40,265
50,136
60,99
70,188
80,452
90,326
100,233
110,184
120,118
130,84
140,270
150,422
160,379
170,223
180,217
190,53
200,169
210,367
220,415
230,214
240,102
250,132
260,257
270,324
280,134
290,95
300,118
310,250
320,342
330,317
340,204
350,192
360,0
452


In [39]:
write_plot(df_train, "train")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
df_val = get_df(val_data)
df_val.describe()

(290, 2)


,bb_size,orientation
count,290.000000,290.000000
mean,209.980453,188.337692
std,48.134571,99.073692
min,110.308655,0.048549
25%,174.573471,105.933453
50%,205.017067,190.254204
75%,239.940842,267.880341
max,349.847107,359.362915


In [41]:
write_plot(df_val, "val")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …